In [1]:
import json
from typing import Any, Dict

# ---------------------------
# 1) Define tool schemas
# ---------------------------
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather for a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "city": {"type": "string", "description": "City name, e.g., 'Sydney'"},
                    "unit": {"type": "string", "enum": ["C", "F"], "description": "Temperature unit"}
                },
                "required": ["city", "unit"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calc",
            "description": "Basic calculator.",
            "parameters": {
                "type": "object",
                "properties": {
                    "op": {"type": "string", "enum": ["add", "sub", "mul", "div"]},
                    "a": {"type": "number"},
                    "b": {"type": "number"}
                },
                "required": ["op", "a", "b"],
                "additionalProperties": False
            }
        }
    }
]

# ---------------------------
# 2) Implement the tools
# ---------------------------
def get_weather(city: str, unit: str) -> Dict[str, Any]:
    demo = {
        "Sydney": {"C": {"temp": 18.0, "cond": "partly cloudy"},
                   "F": {"temp": 64.4, "cond": "partly cloudy"}},
        "Hanoi":  {"C": {"temp": 30.0, "cond": "humid"},
                   "F": {"temp": 86.0, "cond": "humid"}}
    }
    return demo.get(city, {unit: {"temp": 21.0, "cond": "clear"}})[unit]

def calc(op: str, a: float, b: float) -> Dict[str, Any]:
    if op == "add": out = a + b
    elif op == "sub": out = a - b
    elif op == "mul": out = a * b
    elif op == "div": out = a / b if b != 0 else float("inf")
    else: raise ValueError("unknown op")
    return {"result": out}

# ---------------------------
# 3) Minimal “model” adapter
# ---------------------------
class DummyLLM:
    """
    This dummy stands in for a real LLM that supports tool/function calling.
    Replace this with your provider's SDK (keeping the same interface):
      - send(messages, tools) -> returns either plain text OR a tool call
      - send(messages + tool_result) -> returns the final text
    """

    def send(self, messages, tools):
        user = messages[-1]["content"].lower()

        # Naive heuristics to demonstrate: in reality the model decides this.
        if "weather" in user:
            return {
                "role": "assistant",
                "tool_call": {
                    "name": "get_weather",
                    "arguments": {"city": "Sydney", "unit": "C"}
                }
            }
        if "add" in user or "sum" in user or "calculate" in user:
            return {
                "role": "assistant",
                "tool_call": {
                    "name": "calc",
                    "arguments": {"op": "add", "a": 12, "b": 30}
                }
            }
        # Otherwise just answer
        return {"role": "assistant", "content": "I don't need a tool for that."}

    def send_with_tool_result(self, messages):
        # After we provided the tool result, the model produces a final NL answer.
        tool_result = messages[-1]["content"]
        return {
            "role": "assistant",
            "content": f"Here you go: {tool_result}"
        }

# ---------------------------
# 4) Agent loop
# ---------------------------
def run_agent(user_message: str):
    llm = DummyLLM()
    messages = [{"role": "user", "content": user_message}]

    # Step A: ask model; it may choose a tool
    first = llm.send(messages, TOOLS)

    if "tool_call" not in first:
        # No tool needed
        return first["content"]

    # Step B: dispatch tool
    call = first["tool_call"]
    name, args = call["name"], call["arguments"]

    if name == "get_weather":
        result = get_weather(**args)
    elif name == "calc":
        result = calc(**args)
    else:
        result = {"error": f"Unknown tool {name}"}

    # Step C: feed tool result back to the model
    messages.append(first)  # the model’s tool-call message
    messages.append({
        "role": "tool",
        "name": name,
        "content": json.dumps(result)  # tools usually return JSON strings
    })
    final = llm.send_with_tool_result(messages)
    return final["content"]


In [2]:
print(run_agent("What is the weather right now?"))
print(run_agent("Please calculate the sum of 12 and 30."))
print(run_agent("Say hello without tools."))

Here you go: {"temp": 18.0, "cond": "partly cloudy"}
Here you go: {"result": 42}
I don't need a tool for that.
